In [ ]:
# edge detection, creating polygons, and adding polygon characteristics to csv 

In [ ]:
import json
import cv2, matplotlib
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame, read_csv
import pandas as pd

import Image
import math
from PIL import Image,ImageDraw
from PIL import ImageStat
import os.path
from scipy import misc
from skimage import color
import geojson

from skimage import measure
from geojson import MultiPolygon
from geojson import Feature,FeatureCollection


In [ ]:
def brightness( im ):
   status = ImageStat.Stat(im)
   #print status
   r,g,b = status.rms
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))


In [ ]:
def rgb_to_hsv(image):
    hsv= matplotlib.colors.rgb_to_hsv(image)
    average= np.average(hsv, axis=0)
    average_again = np.average(average, axis=0)
    std=np.std(hsv,axis=0)
    std_again=np.std(std,axis=0)

    return (average_again,std_again)


In [ ]:
def output_contour_image(image,polygon):

   # read image as RGB and add alpha (transparency)
   im = Image.open(image).convert("RGBA")

   # convert to numpy (for convenience)
   imArray = np.asarray(im)

   # create mask

   maskIm = Image.new('L', (imArray.shape[1], imArray.shape[0]), 0)
   ImageDraw.Draw(maskIm).polygon(polygon, outline=1, fill=1)
   mask = np.array(maskIm)

   # assemble new image (uint8: 0-255)
   newImArray = np.empty(imArray.shape,dtype='uint8')

   # colors (three first columns, RGB)
   newImArray[:,:,:3] = imArray[:,:,:3]

   # transparency (4th column)
   newImArray[:,:,3] = mask*255

   # back to Image from numpy
   newIm = Image.fromarray(newImArray, "RGBA")

   background = Image.new("RGB", newIm.size, (0, 0, 0))
   background.paste(newIm, mask=newIm.split()[3]) # 3 is the alpha channel
   return background


In [ ]:
#define input output csv files
inputcsv = 'test.csv'
outputcsv='output.csv'

#exception handling
if(outputcsv.find('.csv') == -1):
    raise ValueError('Output has to be a csv file. It has to end with .csv. Abort ')
if(inputcsv.find('.csv') == -1):
    raise ValueError('Input has to be a csv file. It has to end with .csv. Abort ')

#open csv and get the list of images
df=pd.read_csv(inputcsv)
image_list=df['#img'].tolist()


In [ ]:
#calculate different properties
for x in image_list:
    img='images/'+str(x)+'.jpg'
    print img
    contour_list=[]
    outputcsv=x+'/'+x+'.csv'
    if not os.path.exists(str(x)):
       os.makedirs(str(x))

    #check to make sure it is a valid file
    if(not os.path.exists(img)):
        raise ValueError('The file does not exist. Please recheck your list in csv. Abort.')
        
    fimg = misc.imread(img)
    gimg = color.colorconv.rgb2grey(fimg)
    contours = measure.find_contours(gimg, 0.8)
    for n, contour in enumerate(contours):
        element=[]
        for z in contour:
            t=(z[1],z[0])
            element.append(t)
            
        contour_list.append(element)
    count=1   
    
    #csv files
    bright_list=[]
    geoj=[]
    hue_average=[]
    SD_hue=[]
    saturation_average=[]
    SD_saturation=[]
    value_average=[]
    SD_value=[]
    multi_pol=[]
    ID_C=1
    
    for c in contour_list:
        result=output_contour_image(img,c)
        result.save(''+str(x)+'/'+'countour_'+str(count)+'.jpg')
        
        
        my_poly=geojson.Polygon([c])
        my_feature = Feature(geometry=my_poly,id=ID_C)
        multi_pol.append(my_feature)

        
        with open(''+x+'/'+'countour_'+str(count)+'.geojson', 'w') as outfile:
            geojson.dump(my_poly,outfile)
            
        bright_list.append(brightness(result))
        geoj.append(ID_C)
        ID_C=ID_C+1        
        
        hsv_result= rgb_to_hsv(result)
        hue_average.append(hsv_result[0][0])
        saturation_average.append(hsv_result[0][1])
        value_average.append(hsv_result[0][2])
    
        SD_hue.append(hsv_result[1][0])
        SD_saturation.append(hsv_result[1][1])
        SD_value.append(hsv_result[1][2])
        count=count+1
        
        
    mul_p=FeatureCollection(multi_pol)
    with open(''+str(x)+'/'+'all_polygons.geojson', 'w') as outfile:
        geojson.dump(mul_p,outfile)
        
    brt = pd.DataFrame({'brightness': bright_list})
    g = pd.DataFrame({'GeoJson Object id': geoj}) 
    hue_average=[x*255 for x in hue_average]
    SD_hue=[x*255 for x in SD_hue]
    saturation_average=[x*255 for x in saturation_average]
    SD_saturation=[x*255 for x in SD_saturation]
    h_a=pd.DataFrame({'hue_average': hue_average})
    sat_a=pd.DataFrame({'saturation_average': saturation_average})
    val_a=pd.DataFrame({'value_average': value_average})
    h_sd=pd.DataFrame({'hue_STD': SD_hue})
    sat_sd=pd.DataFrame({'saturation_STD': SD_saturation})
    val_sd=pd.DataFrame({'value_STD': SD_value})
    
    result1 = pd.concat([g,brt,sat_sd,sat_a,val_a,val_sd,h_a,h_sd], axis=1)
    result1.to_csv(outputcsv, index = False)